In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
data_dir = 'kanker/Training/'
# Tentukan ukuran gambar yang digunakan
image_size = (128, 128)  # Ukuran gambar yang diresize
batch_size = 32  # Ukuran batch untuk training

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)

# Membaca gambar dan membagi data menjadi training
train_generator = train_datagen.flow_from_directory(
    os.path.join(data_dir),
    target_size=image_size,  # Mengubah ukuran gambar
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=True 
)

# Menampilkan informasi tentang dataset
print(train_generator.class_indices)

Found 5712 images belonging to 4 classes.
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


# Training

In [4]:
# Membangun model CNN
model = tf.keras.Sequential([
    # Layer Konvolusi 1
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Layer Konvolusi 2
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Layer Konvolusi 3
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    # Flatten hasil konvolusi
    tf.keras.layers.Flatten(),

    # Fully Connected Layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Regularisasi dengan Dropout

    # Output Layer (3 kelas: glioma, nontumor, meningioma)
    tf.keras.layers.Dense(4, activation='softmax')  # Softmax untuk multi-class classification
])

# Kompilasi model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # Karena klasifikasi multi-kelas
    metrics=['accuracy']
)

# Ringkasan Model
model.summary()


C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 4)                   │             516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,156 (12.61 MB)

 Trainable params: 3,305,156 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=50  # Bisa disesuaikan
)

C:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 26s 141ms/step - accuracy: 0.6173 - loss: 0.8839
Epoch 2/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 34us/step - accuracy: 0.8438 - loss: 0.3275  
Epoch 3/50


C:\Users\User\anaconda3\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


178/178 ━━━━━━━━━━━━━━━━━━━━ 28s 156ms/step - accuracy: 0.8450 - loss: 0.4247
Epoch 4/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 17us/step - accuracy: 0.9062 - loss: 0.1802  
Epoch 5/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 31s 171ms/step - accuracy: 0.8944 - loss: 0.2917
Epoch 6/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 17us/step - accuracy: 1.0000 - loss: 0.0629  
Epoch 7/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 31s 173ms/step - accuracy: 0.9294 - loss: 0.2001
Epoch 8/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 23us/step - accuracy: 1.0000 - loss: 0.0884  
Epoch 9/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 32s 178ms/step - accuracy: 0.9411 - loss: 0.1568
Epoch 10/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 17us/step - accuracy: 1.0000 - loss: 0.0875  
Epoch 11/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 35s 197ms/step - accuracy: 0.9552 - loss: 0.1322
Epoch 12/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 17us/step - accuracy: 0.9062 - loss: 0.1968  
Epoch 13/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 35s 196ms/step - accuracy: 0.9683 - loss: 0.0999
Epoch 14/50
178/178 ━━━━━━━━

# Testing

In [6]:
data_dir_testing = 'kanker/Testing/'
image_size_testing = (128, 128)  # Ukuran gambar yang diresize
batch_size_testing = 32  # Ukuran batch untuk testing

In [7]:
# ImageDataGenerator untuk normalisasi gambar
test_datagen = ImageDataGenerator(rescale=1./255)

# Membaca gambar dari folder testing dan mempersiapkan data untuk evaluasi
test_generator = test_datagen.flow_from_directory(
    data_dir_testing,  # Direktori data testing
    target_size=image_size_testing,  # Mengubah ukuran gambar
    batch_size=batch_size_testing,
    class_mode='categorical',  # Karena kita memiliki 3 kelas
    shuffle=False  # Tidak perlu mengacak data saat evaluasi
)
print(test_generator.class_indices)

Found 1311 images belonging to 4 classes.
{'glioma': 0, 'meningioma': 1, 'notumor': 2, 'pituitary': 3}


In [8]:
# Evaluasi model pada data testing
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size_testing)

print(f"Testing Loss: {loss}")
print(f"Testing Accuracy: {accuracy}")

40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 95ms/step - accuracy: 0.9507 - loss: 0.1775
Testing Loss: 0.11998015642166138
Testing Accuracy: 0.9664062261581421


In [13]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Muat gambar baru untuk prediksi
img_path = 'kanker/Testing/notumor/Te-no_0087.jpg'
img = image.load_img(img_path, target_size=(128, 128)) 
img_array = image.img_to_array(img) 
img_array = np.expand_dims(img_array, axis=0)  # Menambah dimensi batch
img_array /= 255.0  

prediction = model.predict(img_array)

# Ambil kelas dengan probabilitas tertinggi
predicted_class = np.argmax(prediction)  # Ambil indeks kelas dengan probabilitas tertinggi
class_names = ['glioma', 'nontumor', 'nontumor', 'pituitary']

# Menampilkan prediksi
print(f"Prediksi kelas: {class_names[predicted_class]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Prediksi kelas: nontumor
